In [38]:
import re
import os
import numpy as np
from glob import glob
mainpath = r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Python'
os.chdir(mainpath)
outfile = r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InAs\PBE_GGA_PAW\NoDiag\2ndRelax\InAs_Bulk_PAW.out'
rootpath = r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InAs\PAW_K-pts_PW'
print ("test")
from matplotlib import pyplot as plt
from pylab import MaxNLocator


#  %matplotlib inline

test


In [41]:
alloutputfiles = [ y for x in os.walk(rootpath) for y in glob(os.path.join(x[0],'*.out')) ]

print ( [x[1] for x in os.walk(rootpath)])
#tbh no idea how this works
print (alloutputfiles)

[['Kpts-10', 'Kpts-12', 'Kpts-8'], ['_ecut_20.00Ry', '_ecut_25.00Ry', '_ecut_30.00Ry', '_ecut_35.00Ry', '_ecut_40.00Ry', '_ecut_45.00Ry', '_ecut_50.00Ry'], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['_ecut_20.00Ry', '_ecut_25.00Ry', '_ecut_30.00Ry', '_ecut_35.00Ry', '_ecut_40.00Ry', '_ecut_45.00Ry', '_ecut_50.00Ry'], ['InAsBulk_PAW.save'], [], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['_ecut_20.00Ry', '_ecut_25.00Ry', '_ecut_30.00Ry', '_ecut_35.00Ry', '_ecut_40.00Ry', '_ecut_45.00Ry', '_ecut_50.00Ry'], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], [], ['InAsBulk_PAW.save'], []]
['D:\\Documents\\SivaLab\\2016 MDA Type 2 SL\\Ab-Initio\\B

In [5]:
def calcOverview(outputfile, verbose = True, GetDOS = False):
    # This quickly looks through a QE output file and generates a calculational summary.
    # Outputs:
    # Ionic parts, a list of lists of ionic energies. for a static calc, just a list of length 1 of all electornic steps
    RelaxationSteps = []
    #   Final unit cell pressure
    #   Final Free Energy eV, and Energy/atom and total formation energy
    #   atomic structures. Lattice vectors and atomic positions. List of these for each ionic step.
    atomicStructure = []
    #   Final Structure Volume, lattice vector lengths
    #   Number of atoms
    #   Calculation Time (seconds?)
    #   Execution Date
    calctime = 0
    #   Density of States Optional
    #   K-points
    #   Job completion
    # This summary dictionary will have each ionic step as an array element.
    ionstep = 0
    electronstep = 99
    jobComplete = False
    kpts = 0
    cutoff = 0
    if verbose : print ("\n--Now parsing file " + outputfile)
    with open(outputfile) as fileobj:
        for line in fileobj :
            regexlist = {'iteration' : r'iteration #[\s]+[0-9]+',
                         'total energy' : r'[\s]*total energy[\s]*=[\s]*[\-\.0-9]*',
                         'cutoff' : r'kinetic-energy cutoff[\s]*=[\s]*[\-\.0-9]*',
                         'done' : r'JOB DONE.',
                         'Cell Parameters' : r'CELL_PARAMETERS',
                         'k-points' : r'k points=[\s]*[0-9]*',
                         'exitcode' : r'Exit code:[\s0-9]*'}
            for regex in regexlist :
                searchresult = re.findall(regexlist[regex],line)
                if searchresult :# isn't empty..
                    if verbose: print ("We have a match! :" +(searchresult[0]))
                    if 'iteration' in regex:
                        newstep = int(searchresult[0].split('#')[1])
                        if verbose: print ('electron step #' + str(newstep) + " last step was " + str(electronstep))
                        if electronstep > newstep : #reset to new ionic step
                            ionstep += 1
                            if verbose: print ("looks like a new ion step" + str(ionstep))
                            RelaxationSteps.append([])
                        electronstep = newstep
                    elif 'total energy' in regex : #total energy
                        newenergy = float(searchresult[0].split('=')[1])
                        if verbose: print ("\tthis step energy : " + str(newenergy))
                        RelaxationSteps[ionstep-1].append(newenergy)  #ionstep starts at 1, index starts at 0
                    elif 'Cell Parameters' in regex:
                        lattice = []
                        atoms = []
                        #fileobj.readline()  #read the rest of this line
                        for i in range(3):
                            lattice.append(fileobj.readline().split())
                            if verbose : print (lattice)
                        fileobj.readline()
                        fileobj.readline()  #read the empty line and Atomic_Positions
                        while True: # is not null
                            newatom = fileobj.readline().split()
                            if verbose : print ("new atom: " + str(newatom))
                            if not newatom or newatom[0] == 'End':  #if empty
                                break
                            atoms.append(newatom)
                        atomicStructure = [lattice,atoms]
                    elif 'cutoff' in regex:
                        cutoff = float(searchresult[0].split('=')[1]) # in Ry
                    elif 'k-points' in regex:
                        kpts = float(searchresult[0].split('=')[1]) # total # of kpts
                    elif 'exitcode' in regex:
                        if verbose : print ("Timeout detected! This calculation did not complete! "+ searchresult[0])
                        jobComplete = False
                    elif 'done' in regex:
                        jobComplete = True
                # break  #don't check any more regex if empty
    if verbose :
        if jobComplete:
            completestring = "completed"
        else:
            completestring = "did not complete"
        print ("\n\n--This calculation " + completestring)
        print ("The cutoff is " + str(cutoff))
        print ("number of ionic steps is " + str(ionstep))
        print ("Here's the giant Relax steps" + str(RelaxationSteps))
        print("Final Energy: " + str(RelaxationSteps[-1][-1]))
    return {'RelaxationSteps': RelaxationSteps, 'AtomicStructure' : atomicStructure, 'Total # of K-points' : kpts, 'Cutoff (Ry)' : cutoff, 'Job Complete?': jobComplete}

In [6]:
resultslist = calcOverview(r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InAs\PAW_Kpts_2\kpts-10\cutoff=27.5\InAs_Bulk_PAW.out',False)
    
for stuff in resultslist:
    print (stuff)
    print ('\n')

[['3.068267919', '3.068267919', '-0.000024164'], ['3.067737822', '-0.000186635', '3.067769581'], ['-0.000186635', '3.067737822', '3.067769581']]


[['In', '0.000452896', '0.000013122', '0.000013122'], ['As', '0.250547104', '0.249986878', '0.249986878']]




NameError: name 'num_atoms' is not defined

In [70]:
x = []
y = []
plots = []
kptslist = []
for calcfile in alloutputfiles:
    resultslist = calcOverview(calcfile,False)
    if resultslist['Job Complete?']:
        newx = resultslist['Cutoff (Ry)']
        newy = (resultslist['RelaxationSteps'][-1][-1] / -2 )
        try: 
            index = kptslist.index(resultslist['Total # of K-points'])
            plots[index][0].append(newx)
            plots[index][1].append(newy)
        except (Exception) as exc:
            kptslist.append(resultslist['Total # of K-points'])
            plots.append([[newx],[newy],resultslist['Total # of K-points']])
            print (exc)
print (plots)
fig = plt.figure()
ax = fig.add_subplot(111)

for plot in plots:
    ax.plot(plot[0],plot[1], marker='o', label = plot[2])

y_fmt = tick.FormatStrFormatter('%2.5e')

ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))

plt.show()

282.0 is not in list
476.0 is not in list
150.0 is not in list
[[[20.0, 25.0, 35.0, 40.0, 45.0, 50.0], [293.29325621, 293.38314726, 293.39666538, 293.398384475, 293.399327805, 293.39951754], 282.0], [[20.0, 30.0, 35.0, 40.0, 45.0, 50.0], [293.29335684, 293.39537232, 293.39671126, 293.39841038, 293.39935241, 293.399542385], 476.0], [[40.0, 45.0, 50.0], [293.39828461, 293.39922934, 293.39941775], 150.0]]


In [87]:
# Dictionaries pass references
dictionary = {'one': 1, 'two':2, 'three':3}
result = []
a=5
b=3
a = [1,2,3]
b = a
a = [4,5,6]
print(b,a)
d = [i+1 for i in a]
c = a.copy()
c[1]+=1
print(c,a)
for n in np.linspace(25,40,10):
    newdict = dictionary.copy()
    newdict['one'] = n
    result.append(newdict)
print (result)

[1, 2, 3] [4, 5, 6]
[4, 6, 6] [4, 5, 6]
[{'two': 2, 'one': 25.0, 'three': 3}, {'two': 2, 'one': 26.666666666666668, 'three': 3}, {'two': 2, 'one': 28.333333333333332, 'three': 3}, {'two': 2, 'one': 30.0, 'three': 3}, {'two': 2, 'one': 31.666666666666668, 'three': 3}, {'two': 2, 'one': 33.333333333333336, 'three': 3}, {'two': 2, 'one': 35.0, 'three': 3}, {'two': 2, 'one': 36.666666666666671, 'three': 3}, {'two': 2, 'one': 38.333333333333336, 'three': 3}, {'two': 2, 'one': 40.0, 'three': 3}]


In [56]:
# how to handle exceptions
list1 = [1,2,3,4,4, 5]
try :
    print (list1.index(4))
except (Exception) as e:
    print (e)

3


In [38]:
# change plot y axis
import matplotlib.ticker as tick
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,y, marker = 'o')

#y_fmt = tick.FormatStrFormatter('%2.5e')

#ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))

plt.show()

In [16]:
#Example on how to send commands to the terminal


os.system("dir")  #sends a command to the terminal

# lets you read the command back
with os.popen("dir") as f:
    print (f.readlines())

[' Volume in drive D is Data\n', ' Volume Serial Number is 9C9F-27F7\n', '\n', ' Directory of D:\\Users\\Chris\\Documents\n', '\n', '2016-11-02  03:17 PM    <DIR>          .\n', '2016-11-02  03:17 PM    <DIR>          ..\n', '2016-11-02  03:15 PM    <DIR>          .ipynb_checkpoints\n', '2016-08-22  03:39 PM    <DIR>          Add-in Express\n', '2016-08-22  03:39 PM    <DIR>          Custom Office Templates\n', '2016-08-22  03:39 PM    <DIR>          Heroes of the Storm\n', '2016-09-07  04:29 PM    <DIR>          HyperSpec\n', '2016-10-24  03:13 PM    <DIR>          Jupyter ipython notebooks\n', '2016-08-22  03:39 PM    <DIR>          LabVIEW Data\n', '2016-11-03  09:38 AM    <DIR>          Nyx\n', '2016-08-24  09:05 AM    <DIR>          OneNote Notebooks\n', '2016-10-06  10:27 AM    <DIR>          Outlook Files\n', '2016-10-05  01:27 PM           483,328 putty.exe\n', '2016-08-22  03:39 PM    <DIR>          Python Scripts\n', '2016-08-22  03:39 PM    <DIR>          QDIP SED\n', '2016-

In [113]:
import os
path = r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InSb\PBE_GGA_PAW\InSb_Bulk_PAW - Copy.in'
basepath = os.path.split(path)[0]
os.mkdir(basepath + os.path.sep +'newthing')
print (os.path.join(basepath,'test','test2','test3'))
print (os.path.split(path)[0]+os.path.sep + 'newfile.in')

D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InSb\PBE_GGA_PAW\test\test2\test3
D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InSb\PBE_GGA_PAW\newfile.in


In [10]:
import os
import re
import numpy as np
def makenewcalc(basefile, kpts, cutoff):
    with open(basefile, 'r') as fileobj:
        mainfolder = os.path.split(basefile)[0]
        newpath = os.path.join(mainfolder,'kpts-'+kpts[:2].strip(),'cutoff='+ str(cutoff))
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        newfile = os.path.join(newpath, os.path.split(basefile)[1])
        with open(newfile, 'w') as newfileobj:
            for line in fileobj:
                searchresult = re.findall(r'ecutwfc[\s]+=',line)
                if searchresult: #isn't null
                    newfileobj.write('    ecutwfc         = ' + str(cutoff)+' ,! autogen \n')
                else: 
                    searchresult = re.findall(r'K_POINTS',line)
                    if searchresult: # isn't null again
                        newfileobj.write(line)
                        newfileobj.write('  ' + kpts)
                        fileobj.readline()
                    else:
                        newfileobj.write(line)
basefile = r'D:\Documents\SivaLab\2016 MDA Type 2 SL\Ab-Initio\Bulk\InSb\PBE_GGA_PAW\InSb_Bulk_PAW.in'
kpointslist = ['8 8 8 0 0 0','10 10 10 0 0 0', '12 12 12 0 0 0']
for kpts in kpointslist:
    for cutoff in np.linspace(25,50,6):
        makenewcalc(basefile, kpts,cutoff)